# Binary Image Processing From Scratch

## Introduction
Binary image processing is a fundamental technique in computer vision that involves converting grayscale or color images into a binary (black and white) format. While this transformation might seem simple, it serves as the foundation for many advanced computer vision tasks and has numerous practical applications.

In this notebook, we'll implement various thresholding algorithms from scratch using pure NumPy, which will help us understand the mathematical and algorithmic foundations of binary image processing. Rather than relying on pre-built functions, we'll create our own implementations to gain deeper insights into how these algorithms work.

## Key Concepts Covered
- Understanding image thresholding fundamentals
- Implementing different thresholding techniques from scratch
- Comparing and analyzing various methods
- Exploring real-world applications

## Dependencies
We'll use minimal dependencies to focus on understanding core concepts:
- NumPy: For numerical computations
- Matplotlib: For visualization
- PIL (Python Imaging Library): For image loading

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import random

# Configure matplotlib for better display in notebook
plt.style.use('seaborn')
%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)

## Helper Functions

First we need to create some utility functions that will help develop and test our algoirthms:
1. Load and preprocess images
2. Display results for comparison
3. Analyze image characteristics

In [48]:
def load_image(image_path):
    """Load and convert an image to grayscale.
    Returns:
        numpy.ndarray: Grayscale image array with values 0-255
    """
    try:
        with Image.open(image_path) as img:
            if img.mode != 'L':
                img = img.convert('L')
            # Convert to numpy array and ensure correct dtype
            return np.array(img, dtype=np.uint8)
    except Exception as e:
        raise ValueError(f"Error processing image: {str(e)}")

def display_images(images, titles=None, figsize=(15, 5), cmap='gray',border_color='black', border_width=1):
    """Display images in a truly compact grid layout with maximum 4 images per row.
    
    This function creates a highly compact display by carefully controlling both the
    figure size and the spacing between subplots. It uses a combination of GridSpec
    and figure size calculations to ensure images are displayed with minimal gaps
    while maintaining readability.
    
    Args:
        images (list): List of images to display
        titles (list): Optional list of titles for each image
        figsize (tuple): Base figure size (width, height) used as a reference for calculations
        cmap (str): Colormap to use for displaying images
    """
    n = len(images)
    if titles is None:
        titles = [f'Image {i+1}' for i in range(n)]
    
    # Calculate grid dimensions
    cols = min(n, 4)
    rows = (n + cols - 1) // cols
    
    # Calculate figure size differently to ensure compact layout
    # Use the aspect ratio of the first image to maintain proportions
    sample_image = images[0]
    aspect_ratio = sample_image.shape[0] / sample_image.shape[1]
    
    # Base width calculation considering the number of columns
    base_unit = 4  # Base unit for size calculations
    fig_width = base_unit * cols
    fig_height = base_unit * aspect_ratio * rows
    
    # Create figure and GridSpec with tight spacing
    fig = plt.figure(figsize=(fig_width, fig_height))
    gs = plt.GridSpec(rows, cols, figure=fig,
                     left=0.01, right=0.99,
                     bottom=0.01, top=0.90,
                     wspace=0.01, hspace=0.15)
    
    # Create and populate subplots
    for idx in range(n):
        row = idx // cols
        col = idx % cols
        
        # Create subplot
        ax = fig.add_subplot(gs[row, col])
        
        # Display image without interpolation for sharper display
        ax.imshow(images[idx], cmap=cmap, interpolation='nearest')
        ax.set_title(titles[idx], pad=2, fontsize=14)
        
        # Remove all axes elements
        ax.set_xticks([])
        ax.set_yticks([])
        for spine in ax.spines.values():
                    spine.set_visible(True)  # Make sure spine is visible
                    spine.set_color(border_color)  # Set border color
                    spine.set_linewidth(border_width)  # Set border width
    
    plt.show()

def analyze_image(image):
    """Analyze basic characteristics of an image.
    
    Computes and returns useful statistics about the image:
    - Min and max pixel values
    - Mean and median intensity
    - Standard deviation of pixel values
    - Histogram data
    
    Returns:
        dict: Dictionary containing image statistics
    """
    stats = {
        'min': int(np.min(image)),
        'max': int(np.max(image)),
        'mean': float(np.mean(image)),
        'median': float(np.median(image)),
        'std': float(np.std(image))
    }
    
    # Compute histogram
    hist, bins = np.histogram(image.flatten(), bins=256, range=[0, 256])
    stats['histogram'] = hist
    stats['bins'] = bins
    
    return stats

def plot_histogram(image, title="Image Histogram", figsize=(10, 4)):
    """Plot the histogram of a grayscale image.
    
    Creates a visualization of the image's intensity distribution,
    which is crucial for understanding how to set thresholds.
    
    Args:
        image (numpy.ndarray): Input grayscale image
        title (str): Title for the plot
        figsize (tuple): Figure size
    """
    plt.figure(figsize=figsize)
    plt.hist(image.ravel(), bins=256, range=[0, 256], density=True, alpha=0.7)
    plt.title(title)
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.show()

def process_random_image(folder_path, algorithms, num_images=1, seed=None):
    """Process random images from a directory using specified algorithms.
    
    This function streamlines our testing workflow by:
    1. Randomly selecting images from a directory
    2. Applying multiple processing algorithms
    3. Displaying results side by side for comparison
    
    Args:
        folder_path (str): Path to the directory containing images
        algorithms (dict): Dictionary of {name: function} pairs for processing
                         Each function should take an image array as input
        num_images (int): Number of random images to process
        seed (int): Random seed for reproducibility
    
    Returns:
        list: List of processed image sets for further analysis if needed
    
    Example usage:
        algorithms = {
            'Basic Threshold': lambda img: basic_threshold(img, 127),
            'Adaptive': lambda img: adaptive_threshold(img)
        }
        process_random_image('../images', algorithms)
    """
    # Set random seed if provided
    if seed is not None:
        random.seed(seed)
    
    # Get list of valid image files
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    image_files = [
        f for f in os.listdir(folder_path) 
        if f.lower().endswith(valid_extensions)
    ]
    
    if not image_files:
        raise ValueError(f"No valid images found in {folder_path}")
    
    # Process specified number of random images
    results = []
    for _ in range(num_images):
        image_file = random.choice(image_files)
        image_path = os.path.join(folder_path, image_file)
        
        original = load_image(image_path)
        
        # Apply each algorithm
        processed_images = [original]
        titles = ['Original']
        
        for name, algo in algorithms.items():
            try:
                processed = algo(original)
                processed_images.append(processed)
                titles.append(name)
            except Exception as e:
                print(f"Error applying {name}: {str(e)}")
        
        print(f"\nProcessing: {image_file}")
        
        plot_histogram(original, f"Histogram for {image_file}")
        
        display_images(
            processed_images,
            titles,
            figsize=(4 * len(processed_images), 4)
        )
        
        stats = analyze_image(original)
        print("\nImage Statistics:")
        print(f"Dimensions: {original.shape}")
        print(f"Intensity Range: {stats['min']} - {stats['max']}")
        print(f"Mean Intensity: {stats['mean']:.2f}")
        print(f"Standard Deviation: {stats['std']:.2f}")
        
        results.append({
            'filename': image_file,
            'original': original,
            'processed': processed_images[1:],
            'stats': stats
        })
    
    return results   

# Image Thresholding Algorithms Implementation

We'll implement several thresholding algorithms, each with its own strengths and ideal use cases. For each algorithm, we'll:
1. Explain the mathematical foundation
2. Implement the algorithm from scratch
3. Demonstrate its behavior on example images
4. Discuss its advantages and limitations

## Algorithm Categories

Our implementations cover the main approaches to image thresholding:

1. **Global Thresholding**: Uses a single threshold value for the entire image
2. **Adaptive Thresholding**: Calculates different thresholds for different image regions
3. **Statistical Methods**: Uses image statistics to determine optimal thresholds
4. **Information Theory Methods**: Leverages entropy and information content
5. **Geometric Methods**: Uses geometric properties of the histogram

Each method addresses different challenges in image segmentation.

In [49]:
def simple_threshold(image, threshold, high_value=255, low_value=0):
    """Implement simple global thresholding.
    
    This is the most basic form of thresholding where we use a single global
    threshold value. Pixels above this value become foreground (high_value),
    while pixels below become background (low_value).

    """
    if not isinstance(image, np.ndarray):
        raise TypeError("Input image must be a numpy array")
        
    if image.dtype != np.uint8:
        raise ValueError("Image must be 8-bit (dtype=uint8)")
    
    return np.where(image > threshold, high_value, low_value).astype(np.uint8)

def mean_adaptive_threshold(image, window_size=11, c=2):
    """Implement mean adaptive thresholding.
    
    This method calculates a threshold for each pixel based on the mean
    of its neighborhood. The constant c is subtracted from the mean to
    fine-tune the threshold.
    
    Mathematical formula:
    T(x,y) = mean(neighborhood) - c
    
    """
    if window_size % 2 == 0:
        raise ValueError("Window size must be odd")

    pad_size = window_size // 2
    padded = np.pad(image, pad_size, mode='reflect')
    
    binary = np.zeros_like(image)
    
    rows, cols = image.shape
    for i in range(rows):
        for j in range(cols):
            neighborhood = padded[i:i + window_size, j:j + window_size]
            threshold = np.mean(neighborhood) - c
            binary[i, j] = 255 if image[i, j] > threshold else 0
            
    return binary

def gaussian_adaptive_threshold(image, window_size=11, sigma=2.0, c=2):
    """Implement Gaussian adaptive thresholding.
    
    Similar to mean adaptive thresholding, but uses a weighted sum where
    pixels closer to the center have more influence on the threshold.
    
    Mathematical formula:
    T(x,y) = (G * f)(x,y) - c
    """
    if window_size % 2 == 0:
        raise ValueError("Window size must be odd")
    
    kernel = create_gaussian_kernel(window_size, sigma)
    
    pad_size = window_size // 2
    padded = np.pad(image, pad_size, mode='reflect')
    
    binary = np.zeros_like(image)
    
    rows, cols = image.shape
    for i in range(rows):
        for j in range(cols):
            neighborhood = padded[i:i + window_size, j:j + window_size]
            threshold = np.sum(neighborhood * kernel) - c
            binary[i, j] = 255 if image[i, j] > threshold else 0
            
    return binary

def create_gaussian_kernel(size, sigma):
    """Create a 2D Gaussian kernel."""
    ax = np.linspace(-(size - 1) / 2., (size - 1) / 2., size)
    xx, yy = np.meshgrid(ax, ax)
    
    kernel = np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(sigma))
    return kernel / np.sum(kernel)

def otsu_threshold(image):
    """Implement Otsu's thresholding method.
    
    Otsu's method works by minimizing the within-class variance and maximizing
    the between-class variance of the two classes (foreground and background).
    
    The algorithm:
    1. Compute histogram and probability distribution
    2. For each possible threshold:
        - Split pixels into two classes
        - Calculate class means and variances
        - Calculate between-class variance
    3. Select threshold that maximizes between-class variance
    
    """
    
    hist, _ = np.histogram(image.flatten(), bins=256, range=[0, 256])
    hist = hist.astype(float)
    hist /= np.sum(hist)
    
    max_variance = 0
    optimal_threshold = 0
    
    cumsum = np.cumsum(hist)
    cumsum_val = np.cumsum(hist * np.arange(256))
    global_mean = cumsum_val[-1]
    
    for threshold in range(1, 255):
        w0 = cumsum[threshold]  # Background
        w1 = 1 - w0  # Foreground
        
        if w0 == 0 or w1 == 0:
            continue
            
        # Class means
        mu0 = cumsum_val[threshold] / w0
        mu1 = (global_mean - cumsum_val[threshold]) / w1
        
        # Calculate between-class variance
        variance = w0 * w1 * (mu0 - mu1) ** 2
        
        if variance > max_variance:
            max_variance = variance
            optimal_threshold = threshold
    
    binary = simple_threshold(image, optimal_threshold)
    return binary, optimal_threshold

def multilevel_threshold(image, num_classes=3):
    """Implement multi-level thresholding using k-means clustering.
    
    This method segments the image into multiple classes rather than just
    foreground and background. It uses k-means clustering on pixel intensities
    to find optimal threshold values.
    
    Args:
        image (numpy.ndarray): Input grayscale image
        num_classes (int): Number of desired classes/segments
        
    Returns:
        tuple: (segmented_image, thresholds)
    """
    # Flatten image and convert to float
    pixels = image.flatten().astype(float)
    
    # Initialize centroids
    centroids = np.linspace(0, 255, num_classes)
    old_centroids = np.zeros_like(centroids)
    
    # Iterate until convergence
    max_iterations = 100
    iteration = 0
    
    while not np.array_equal(old_centroids, centroids) and iteration < max_iterations:
        old_centroids = centroids.copy()
        
        # Assign pixels to nearest centroid
        distances = np.abs(pixels.reshape(-1, 1) - centroids)
        labels = np.argmin(distances, axis=1)
        
        # Update centroids
        for i in range(num_classes):
            if np.sum(labels == i) > 0:
                centroids[i] = np.mean(pixels[labels == i])
                
        iteration += 1
    
    # Create segmented image
    segmented = centroids[labels].reshape(image.shape)
    
    # Calculate thresholds between centroids
    thresholds = np.mean([centroids[:-1], centroids[1:]], axis=0)
    
    return segmented.astype(np.uint8), thresholds

def local_threshold(image, window_size=35, k=0.2):
    """Implement local thresholding using Sauvola's method.
    
    This technique adapts to local image statistics using both mean
    and standard deviation in a window:
    
    T(x,y) = mean(x,y) * (1 + k * ((std(x,y) / R) - 1))
    
    where R is the maximum standard deviation (128 for 8-bit images)
    
    Args:
        image (numpy.ndarray): Input grayscale image
        window_size (int): Size of local window
        k (float): Sensitivity factor (0.2-0.5 typical)
        
    Returns:
        numpy.ndarray: Binary image
    """
    # Pad image
    pad_size = window_size // 2
    padded = np.pad(image, pad_size, mode='reflect')
    
    # Initialize output
    binary = np.zeros_like(image)
    
    # Calculate threshold for each pixel
    rows, cols = image.shape
    R = 128  # Dynamic range for 8-bit images
    
    for i in range(rows):
        for j in range(cols):
            window = padded[i:i + window_size, j:j + window_size]
            
            mean = np.mean(window)
            std = np.std(window)
            
            # Calculate threshold using Sauvola's formula
            threshold = mean * (1 + k * ((std / R) - 1))
            
            binary[i, j] = 255 if image[i, j] > threshold else 0
    
    return binary

def triangle_threshold(image):
    """Implement triangle algorithm for thresholding.
    
    This method:
    1. Finds the peak in the histogram
    2. Creates a line from peak to the last non-zero bin
    3. Finds the point of maximum perpendicular distance from this line
    
    Returns:
        tuple: (binary_image, threshold)
    """
    # Calculate histogram
    hist, bins = np.histogram(image.flatten(), bins=256, range=[0, 256])
    
    # Find peak (mode)
    peak_idx = np.argmax(hist)
    
    # Find last non-zero bin
    last_idx = 255 - np.argmax(hist[::-1] > 0)
    
    # Create line between peak and last point
    x1, y1 = peak_idx, hist[peak_idx]
    x2, y2 = last_idx, hist[last_idx]
    
    # Calculate distances from line to histogram points
    max_dist = 0
    threshold = peak_idx
    
    for i in range(peak_idx, last_idx):
        if y2 - y1 != 0:
            dist = np.abs((y2-y1)*i - (x2-x1)*hist[i] + x2*y1 - y2*x1) / np.sqrt((y2-y1)**2 + (x2-x1)**2)
        else:
            dist = hist[i]
            
        if dist > max_dist:
            max_dist = dist
            threshold = i
    
    binary = simple_threshold(image, threshold)
    return binary, threshold

def entropy_threshold(image):
    """Implement entropy-based thresholding.
    
    This method maximizes the sum of entropies of the foreground
    and background distributions. The optimal threshold occurs where
    the total entropy is maximized.
        
    Returns:
        tuple: (binary_image, threshold)
    """
    # Calculate histogram and probabilities
    hist, _ = np.histogram(image.flatten(), bins=256, range=[0, 256])
    hist = hist.astype(float)
    hist /= np.sum(hist)
    
    max_entropy = 0
    optimal_threshold = 0
    
    # Try each possible threshold
    for threshold in range(1, 255):
        # Split histogram
        p0 = hist[:threshold]
        p1 = hist[threshold:]
        
        if len(p0) == 0 or len(p1) == 0:
            continue
            
        # Normalize probabilities
        p0_norm = p0 / np.sum(p0)
        p1_norm = p1 / np.sum(p1)
        
        # Calculate entropy for both regions
        entropy0 = -np.sum(p0_norm * np.log2(p0_norm + np.finfo(float).eps))
        entropy1 = -np.sum(p1_norm * np.log2(p1_norm + np.finfo(float).eps))
        
        # Calculate total entropy
        total_entropy = entropy0 + entropy1
        
        if total_entropy > max_entropy:
            max_entropy = total_entropy
            optimal_threshold = threshold
    
    binary = simple_threshold(image, optimal_threshold)
    return binary, optimal_threshold

def isodata_threshold(image):
    """Implement ISODATA (Iterative Self-Organizing Data Analysis) algorithm. (AUTO THRESHOLD)
    
    This method iteratively updates the threshold by:
    1. Starting with initial threshold (mean of image)
    2. Computing means of pixels above and below threshold
    3. Computing new threshold as average of these means
    4. Repeating until convergence
       
    """
    # Initialize threshold as mean
    threshold = np.mean(image)
    old_threshold = 0
    
    # Iterate until convergence
    max_iterations = 100
    iteration = 0
    
    while abs(threshold - old_threshold) > 0.5 and iteration < max_iterations:
        # Save old threshold
        old_threshold = threshold
        
        # Get pixels in each class
        foreground = image[image >= threshold]
        background = image[image < threshold]
        
        # Calculate new threshold as mean of class means
        threshold = (np.mean(foreground) + np.mean(background)) / 2
        iteration += 1
    
    binary = simple_threshold(image, threshold)
    return binary, threshold

def median_threshold(image, offset=0):
    """Implement median-based thresholding.
    
    This method uses the median pixel value as the threshold, which is particularly
    effective when the image has a balanced distribution of intensities. The offset
    parameter allows fine-tuning of the threshold relative to the median.
    """

    threshold = np.median(image) + offset
    return (image > threshold).astype(np.uint8) * 255

def quartile_threshold(image, quartile=0.75):
    """Implement quartile-based thresholding.
    
    Uses a specified quartile of pixel intensities as the threshold. This is useful
    when you want to segment based on statistical properties of the intensity
    distribution.
    
    """
    threshold = np.quantile(image, quartile)
    return (image > threshold).astype(np.uint8) * 255

def kmeans_threshold(image, k=2, max_iter=100):
    """Implement k-means based thresholding.
    
    Uses k-means clustering to group pixels into k intensity clusters, then
    uses the boundaries between clusters as thresholds.
    """
    pixels = image.ravel().astype(float)
    
    # Initialize centroids
    min_val, max_val = np.min(pixels), np.max(pixels)
    centroids = np.linspace(min_val, max_val, k)
    
    for _ in range(max_iter):
        old_centroids = centroids.copy()
        
        # Assign pixels to nearest centroid
        distances = np.abs(pixels.reshape(-1, 1) - centroids)
        labels = np.argmin(distances, axis=1)
        
        # Update centroids
        for i in range(k):
            if np.sum(labels == i) > 0:
                centroids[i] = np.mean(pixels[labels == i])
        
        if np.allclose(old_centroids, centroids):
            break
    
    # Sort centroids and find thresholds between them
    centroids.sort()
    thresholds = [(centroids[i] + centroids[i+1])/2 for i in range(k-1)]
    
    # Create binary image using middle threshold if k=2
    binary = (image > thresholds[0]).astype(np.uint8) * 255 if k == 2 else None
    
    return binary, thresholds

def fuzzy_threshold(image, c=2, max_iter=100, m=2):
    """Implement fuzzy c-means thresholding.
    
    Uses fuzzy c-means clustering to handle uncertainty in pixel classification.
    Each pixel can belong to multiple classes with different degrees of membership.
    
    Args:
        image (numpy.ndarray): Input grayscale image
        c (int): Number of clusters
        max_iter (int): Maximum number of iterations
        m (float): Fuzziness parameter
    
    Returns:
        tuple: (binary_image, threshold)
    """
    pixels = image.ravel().astype(float)
    n = len(pixels)
    
    # Initialize membership matrix
    U = np.random.rand(c, n)
    U = U / np.sum(U, axis=0)
    
    for _ in range(max_iter):
        old_U = U.copy()
        
        # Calculate cluster centers
        U_m = U ** m
        centers = np.sum(U_m * pixels, axis=1) / np.sum(U_m, axis=1)
        
        # Update membership matrix
        for i in range(c):
            distances = np.abs(pixels - centers[i])
            for j in range(c):
                distances_j = np.abs(pixels - centers[j])
                U[i] = 1 / np.sum((distances/distances_j)**(2/(m-1)), axis=0)
        
        if np.allclose(old_U, U):
            break
    
    centers.sort()
    threshold = np.mean(centers)
    
    return (image > threshold).astype(np.uint8) * 255, threshold

def wavelet_threshold(image, wavelet='haar', level=1):
    """Implement wavelet-based thresholding.
    
    Uses wavelet transform to analyze image at different scales and determine
    threshold based on wavelet coefficients.
    
    """
    try:
        import pywt
    except ImportError:
        raise ImportError("Please install PyWavelets: pip install PyWavelets")
    
    # Perform wavelet decomposition
    coeffs = pywt.wavedec2(image, wavelet, level=level)
    
    # Calculate threshold using detail coefficients
    detail_coeffs = list(coeffs[1:])
    threshold = np.sqrt(2 * np.log(image.size))
    
    # Apply threshold to coefficients
    threshold_value = np.mean([np.median(np.abs(d)) * threshold for d in detail_coeffs])
    
    return (image > threshold_value).astype(np.uint8) * 255, threshold_value

def bernsen_threshold(image, window_size=15, contrast_threshold=15):
    """Implement Bernsen's local thresholding method.
    
    Uses local contrast in a window to determine threshold. If contrast is too low,
    the pixel is set based on the global threshold.
    
    """
    rows, cols = image.shape
    output = np.zeros_like(image)
    pad = window_size // 2
    
    # Pad image
    padded = np.pad(image, pad, mode='reflect')
    
    for i in range(rows):
        for j in range(cols):
            # Extract window
            window = padded[i:i+window_size, j:j+window_size]
            min_val = np.min(window)
            max_val = np.max(window)
            
            # Calculate local contrast
            contrast = max_val - min_val
            local_threshold = (min_val + max_val) / 2
            
            if contrast < contrast_threshold:
                # Use global threshold if contrast is too low
                output[i, j] = 255 if image[i, j] > np.mean(image) else 0
            else:
                output[i, j] = 255 if image[i, j] > local_threshold else 0
                
    return output

def mean_shift_threshold(image, bandwidth=30):
    """Implement mean-shift based thresholding.
    
    Uses mean-shift clustering to find modes in the intensity distribution
    and determine thresholds.
    
    """
    intensities = image.ravel()
    
    current_means = np.random.choice(intensities, size=10)
    
    # Mean-shift iteration
    for _ in range(20):
        for i in range(len(current_means)):
            distances = np.abs(intensities - current_means[i])
            weights = np.exp(-0.5 * (distances / bandwidth) ** 2)
            
            # Update mean
            current_means[i] = np.sum(intensities * weights) / np.sum(weights)
    
    # Find unique modes
    modes = np.unique(np.round(current_means))
    
    # Use middle value as threshold for binary image
    threshold = np.median(modes)
    
    return (image > threshold).astype(np.uint8) * 255, threshold

def gradient_threshold(image, k=1, sigma=1.0):
    """Implement gradient-based thresholding.
    
    Uses image gradient magnitude to determine threshold. Areas with high
    gradient magnitudes often correspond to edges between regions.

    """
    # Calculate gradients
    gx = np.gradient(image, axis=1)
    gy = np.gradient(image, axis=0)
    
    # Calculate gradient magnitude
    gradient_mag = np.sqrt(gx**2 + gy**2)
    
    # Find threshold using gradient magnitude
    threshold = np.mean(gradient_mag) + k * np.std(gradient_mag)
    
    return (image > threshold).astype(np.uint8) * 255, threshold

In [ ]:
thresholding_algorithms = {
    'Simple (T=127)': lambda image : simple_threshold(image, 127),
    'Mean Adaptive': lambda image : mean_adaptive_threshold(image),
    'Gaussian Adaptive': lambda image :gaussian_adaptive_threshold(image),
    'Otsu': lambda image :otsu_threshold(image)[0],
    'Multilevel': lambda image :multilevel_threshold(image)[0],
    'Local': lambda image :local_threshold(image),
    'Triangle': lambda image :triangle_threshold(image)[0],
    'Entropy': lambda image :entropy_threshold(image)[0],
    'ISODATA': lambda image :isodata_threshold(image)[0],
    'Median': lambda image: median_threshold(image),
    'Quartile': lambda image: quartile_threshold(image),
    'K-means': lambda image: kmeans_threshold(image)[0],
    'Fuzzy': lambda image: fuzzy_threshold(image)[0],
    'Bernsen': lambda image: bernsen_threshold(image),
    'Mean-shift': lambda image: mean_shift_threshold(image)[0],
    'Gradient': lambda image: gradient_threshold(image)[0]
}

# Now we can test these algorithms on random images
def test_thresholding():
    try:
        results = process_random_image(
            '../images',
            thresholding_algorithms,
            num_images=1
        )
        
        for result in results:
            print(f"\nAnalyzing results for {result['filename']}:")
            print("Original image statistics:")
            print(f"Mean intensity: {result['stats']['mean']:.2f}")
            print(f"Standard deviation: {result['stats']['std']:.2f}")
            
    except Exception as e:
        print(f"Error during processing: {str(e)}")

# Run the test
test_thresholding()